In [1]:
import os
import glob
import pandas as pd

In [2]:
file_path = r'C:/Users/Ryan/Documents/BootCamp/Final_Project/Disney_wait_times/Data_Files/'

In [3]:
all_files = glob.glob(os.path.join(file_path, '*.csv'))

In [4]:
wait_time_df = pd.concat((pd.read_csv(f) for f in all_files),)


In [5]:
wait_time_df.head()

,date,datetime,SACTMIN,SPOSTMIN,ride_name
0,1/1/2015,1/1/2015 7:51,NaN,45.0,7_dwarfs_train
1,1/1/2015,1/1/2015 8:02,NaN,60.0,7_dwarfs_train
2,1/1/2015,1/1/2015 8:05,54.0,NaN,7_dwarfs_train
3,1/1/2015,1/1/2015 8:09,NaN,60.0,7_dwarfs_train
4,1/1/2015,1/1/2015 8:16,NaN,60.0,7_dwarfs_train


In [6]:
wait_time_df.dtypes

date          object
datetime      object
SACTMIN      float64
SPOSTMIN     float64
ride_name     object
dtype: object

In [7]:
wait_time_df = wait_time_df.rename(columns={'SACTMIN': 'sactmin', 'SPOSTMIN': 'spostmin'})

In [9]:
wait_time_est = wait_time_df.dropna(subset=['spostmin'])
wait_time_est.head()

,date,datetime,sactmin,spostmin,ride_name
0,1/1/2015,1/1/2015 7:51,NaN,45.0,7_dwarfs_train
1,1/1/2015,1/1/2015 8:02,NaN,60.0,7_dwarfs_train
3,1/1/2015,1/1/2015 8:09,NaN,60.0,7_dwarfs_train
4,1/1/2015,1/1/2015 8:16,NaN,60.0,7_dwarfs_train
6,1/1/2015,1/1/2015 8:23,NaN,60.0,7_dwarfs_train


In [10]:
wait_time_actuals.count()

date         69883
datetime     69883
sactmin      69883
spostmin         0
ride_name    69883
dtype: int64

In [11]:
wait_time_est.count()

date         2345606
datetime     2345606
sactmin            0
spostmin     2345606
ride_name    2345606
dtype: int64

In [12]:
wait_time_est = wait_time_est.drop(columns='sactmin')

In [20]:
wait_time_est = wait_time_est.drop(wait_time_est.loc[wait_time_est['spostmin'] == -999].index)

In [21]:
wait_time_est['spostmin'].describe()

count    1.412810e+06
mean     4.445371e+01
std      3.047277e+01
min      0.000000e+00
25%      2.000000e+01
50%      4.000000e+01
75%      6.000000e+01
max      3.300000e+02
Name: spostmin, dtype: float64

In [22]:
wait_time_est['datetime'] = pd.to_datetime(wait_time_est['datetime'])

In [23]:
wait_time_est.dtypes

date                 object
datetime     datetime64[ns]
spostmin            float64
ride_name            object
dtype: object

In [24]:
wait_time_est['year'] = pd.DatetimeIndex(wait_time_est['datetime']).year

In [25]:
wait_time_est['month'] = pd.DatetimeIndex(wait_time_est['datetime']).month

In [26]:
wait_time_est.head()

,date,datetime,spostmin,ride_name,year,month
6,1/1/2015,2015-01-01 08:23:00,60.0,7_dwarfs_train,2015,1
7,1/1/2015,2015-01-01 08:29:00,60.0,7_dwarfs_train,2015,1
8,1/1/2015,2015-01-01 08:37:00,90.0,7_dwarfs_train,2015,1
9,1/1/2015,2015-01-01 08:44:00,105.0,7_dwarfs_train,2015,1
10,1/1/2015,2015-01-01 08:50:00,105.0,7_dwarfs_train,2015,1


In [27]:
wait_time_est.shape

(1412810, 6)